In [2]:
# ==========================================
# STEP 1: INSTALL & DOWNLOAD DATA
# ==========================================
# 1. INSTALL THE LIBRARY FIRST (This fixes your error)
!pip install opendatasets --quiet

import os
import shutil
import random
import opendatasets as od

# 2. Download Dataset A (Adults/General)
print("⬇️ Downloading COVID-19 Radiography Database (Dataset A)...")
od.download('https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database')

# 3. Download Dataset B (Pediatric Booster)
print("⬇️ Downloading Chest X-Ray Pneumonia (Dataset B)...")
od.download('https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia')

print("\n✅ Downloads Complete! You should see two folders on the left sidebar.")

⬇️ Downloading COVID-19 Radiography Database (Dataset A)...
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Ranjeet0045
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database


100%|██████████| 778M/778M [00:12<00:00, 64.1MB/s]

⬇️ Downloading Chest X-Ray Pneumonia (Dataset B)...
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Ranjeet0045
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia


100%|██████████| 2.29G/2.29G [00:32<00:00, 75.5MB/s]




✅ Downloads Complete! You should see two folders on the left sidebar.


In [3]:
# ==========================================
# STEP 2: MERGE, BALANCE & SPLIT (70/20/10)
# ==========================================
import os
import shutil
import random

# 1. Define Destination
BASE_DIR = 'Final_Dataset'
if os.path.exists(BASE_DIR): shutil.rmtree(BASE_DIR) # Clean reset

# Create folder structure
classes = ['NORMAL', 'PNEUMONIA']
for split in ['train', 'test', 'val']: # Note: test is 20%, val is 10%
    for cls in classes:
        os.makedirs(os.path.join(BASE_DIR, split, cls), exist_ok=True)

print("📂 Created folder structure. Gathering files...")

# 2. Collect All Normal Images (Adults + Kids)
normal_files = []
# From Dataset 1 (Adults)
d1_norm = 'covid19-radiography-database/COVID-19_Radiography_Dataset/Normal'
if os.path.exists(d1_norm):
    normal_files += [os.path.join(d1_norm, f) for f in os.listdir(d1_norm) if f.lower().endswith('png')]

# From Dataset 2 (Kids)
d2_norm_train = 'chest-xray-pneumonia/chest_xray/train/NORMAL'
d2_norm_test = 'chest-xray-pneumonia/chest_xray/test/NORMAL'
if os.path.exists(d2_norm_train):
    normal_files += [os.path.join(d2_norm_train, f) for f in os.listdir(d2_norm_train) if f.lower().endswith('jpeg')]
if os.path.exists(d2_norm_test):
    normal_files += [os.path.join(d2_norm_test, f) for f in os.listdir(d2_norm_test) if f.lower().endswith('jpeg')]

# 3. Collect All Pneumonia Images (Lung Opacity + Viral + COVID + Kids Pneu)
pneumonia_files = []

# D1: Lung Opacity (Bacterial)
d1_opaque = 'covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity'
if not os.path.exists(d1_opaque): d1_opaque = 'covid19-radiography-database/COVID-19_Radiography_Dataset/Lung Opacity'
if os.path.exists(d1_opaque):
    pneumonia_files += [os.path.join(d1_opaque, f) for f in os.listdir(d1_opaque) if f.lower().endswith('png')]

# D1: Viral Pneumonia
d1_viral = 'covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia'
if os.path.exists(d1_viral):
    pneumonia_files += [os.path.join(d1_viral, f) for f in os.listdir(d1_viral) if f.lower().endswith('png')]

# D1: COVID (Treating as Pneumonia for training)
d1_covid = 'covid19-radiography-database/COVID-19_Radiography_Dataset/COVID'
if os.path.exists(d1_covid):
    pneumonia_files += [os.path.join(d1_covid, f) for f in os.listdir(d1_covid) if f.lower().endswith('png')]

# D2: Kids Pneumonia
d2_pneu_train = 'chest-xray-pneumonia/chest_xray/train/PNEUMONIA'
d2_pneu_test = 'chest-xray-pneumonia/chest_xray/test/PNEUMONIA'
if os.path.exists(d2_pneu_train):
    pneumonia_files += [os.path.join(d2_pneu_train, f) for f in os.listdir(d2_pneu_train) if f.lower().endswith('jpeg')]
if os.path.exists(d2_pneu_test):
    pneumonia_files += [os.path.join(d2_pneu_test, f) for f in os.listdir(d2_pneu_test) if f.lower().endswith('jpeg')]

print(f"📊 Found: {len(normal_files)} Normal vs {len(pneumonia_files)} Pneumonia")

# 4. Balance the Classes (Trim the larger one)
if len(pneumonia_files) > len(normal_files):
    print(f"⚖️ Balancing: Trimming Pneumonia from {len(pneumonia_files)} to {len(normal_files)}")
    random.shuffle(pneumonia_files)
    pneumonia_files = pneumonia_files[:len(normal_files)]
elif len(normal_files) > len(pneumonia_files):
    print(f"⚖️ Balancing: Trimming Normal from {len(normal_files)} to {len(pneumonia_files)}")
    random.shuffle(normal_files)
    normal_files = normal_files[:len(pneumonia_files)]

# 5. Distribute Files (70% Train, 20% Test, 10% Val)
def distribute(file_list, category):
    random.shuffle(file_list)
    total = len(file_list)

    # Calculate cut-off points
    train_end = int(0.70 * total)
    test_end = int(0.90 * total) # 70% + 20% = 90%
    # Remaining 10% (from 0.90 to 1.0) goes to Val

    for i, f in enumerate(file_list):
        if i < train_end: split = 'train'
        elif i < test_end: split = 'test'
        else: split = 'val'

        shutil.copy(f, os.path.join(BASE_DIR, split, category, os.path.basename(f)))

print("🚀 Moving files... (This takes ~2-3 minutes)")
distribute(normal_files, 'NORMAL')
distribute(pneumonia_files, 'PNEUMONIA')

print("✅ Data Merged & Split Successfully!")
print("   Check the 'Final_Dataset' folder on the left.")

📂 Created folder structure. Gathering files...
📊 Found: 1575 Normal vs 4265 Pneumonia
⚖️ Balancing: Trimming Pneumonia from 4265 to 1575
🚀 Moving files... (This takes ~2-3 minutes)
✅ Data Merged & Split Successfully!
   Check the 'Final_Dataset' folder on the left.


In [4]:
# ==========================================
# STEP 3: BUILD & TRAIN (BINARY MODEL)
# ==========================================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = 'Final_Dataset'

print("⚙️ Setting up Data Generators...")

# 1. TRAIN GENERATOR (With Augmentation)
# We zoom and rotate images slightly so the model learns to recognize pneumonia from all angles
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    f'{DATA_DIR}/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Binary because we only have Normal vs Pneumonia
    shuffle=True
)

# 2. VALIDATION GENERATOR (No Augmentation, just rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    f'{DATA_DIR}/val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# 3. BUILD THE CNN MODEL
print("🏗️ Building the CNN Model...")
model = Sequential([
    # Block 1
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Block 2
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Block 3
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Deep Features
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Drops 50% of neurons to prevent memorization

    # OUTPUT LAYER: 1 Neuron (0 = Normal, 1 = Pneumonia)
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 4. START TRAINING
print("🚀 Training Started... This will take about 10-15 minutes.")
print("   (You will see the accuracy increase with every Epoch)")

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# 5. SAVE THE MODEL
model.save('pneumonia_binary_model.h5')
print("\n✅ SUCCESS! Model saved as 'pneumonia_binary_model.h5'")

⚙️ Setting up Data Generators...
Found 2204 images belonging to 2 classes.
Found 316 images belonging to 2 classes.
🏗️ Building the CNN Model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🚀 Training Started... This will take about 10-15 minutes.
   (You will see the accuracy increase with every Epoch)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 69s 858ms/step - accuracy: 0.7730 - loss: 2.7987 - val_accuracy: 0.5000 - val_loss: 46.5271
Epoch 2/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 51s 744ms/step - accuracy: 0.8275 - loss: 0.5173 - val_accuracy: 0.4684 - val_loss: 8.3447
Epoch 3/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 51s 736ms/step - accuracy: 0.8644 - loss: 0.3770 - val_accuracy: 0.5000 - val_loss: 19.9300
Epoch 4/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 51s 740ms/step - accuracy: 0.8625 - loss: 0.3394 - val_accuracy: 0.5000 - val_loss: 37.0925
Epoch 5/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 51s 745ms/step - accuracy: 0.8895 - loss: 0.2970 - val_accuracy: 0.5000 - val_loss: 24.6149
Epoch 6/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 52s 759ms/step - accuracy: 0.8718 - loss: 0.3365 - val_accuracy: 0.5063 - val_loss: 5.6056
Epoch 7/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 52s 753ms/step - accuracy: 0.9076 - loss: 0.2780 - val_accuracy: 0.6424 - val_loss: 3.8245
Epoch 8/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 55s 796ms/step - accuracy: 0.9035 - loss: 0.2743 - val_


✅ SUCCESS! Model saved as 'pneumonia_binary_model.h5'


In [6]:
# ==========================================
# NEW STEP 2: MERGE EVERYTHING (NO TRIMMING)
# ==========================================
import os
import shutil
import random
from tqdm import tqdm # Progress bar

BASE_DIR = 'Final_Dataset_Full'
if os.path.exists(BASE_DIR): shutil.rmtree(BASE_DIR)

# Create structure
for split in ['train', 'test', 'val']:
    for cls in ['NORMAL', 'PNEUMONIA']:
        os.makedirs(os.path.join(BASE_DIR, split, cls), exist_ok=True)

print("🕵️ SEARCHING FOR IMAGES...")

# --- 1. DEFINE SOURCES ---
# We map specific source folders to our target classes
sources = [
    # (Path to source folder, Target Class)

    # DATASET A (Adults)
    ('covid19-radiography-database/COVID-19_Radiography_Dataset/Normal', 'NORMAL'),
    ('covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity', 'PNEUMONIA'),
    ('covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia', 'PNEUMONIA'),
    ('covid19-radiography-database/COVID-19_Radiography_Dataset/COVID', 'PNEUMONIA'),

    # DATASET B (Kids) - Train folder
    ('chest-xray-pneumonia/chest_xray/train/NORMAL', 'NORMAL'),
    ('chest-xray-pneumonia/chest_xray/train/PNEUMONIA', 'PNEUMONIA'),

    # DATASET B (Kids) - Test/Val folders (We merge them all to reshuffle)
    ('chest-xray-pneumonia/chest_xray/test/NORMAL', 'NORMAL'),
    ('chest-xray-pneumonia/chest_xray/test/PNEUMONIA', 'PNEUMONIA'),
    ('chest-xray-pneumonia/chest_xray/val/NORMAL', 'NORMAL'),
    ('chest-xray-pneumonia/chest_xray/val/PNEUMONIA', 'PNEUMONIA'),
]

# --- 2. COLLECT FILES ---
normal_files = []
pneumonia_files = []

for path, target_cls in sources:
    if os.path.exists(path):
        # Find all images
        images = [os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        count = len(images)
        print(f"   Found {count} images in: {path} -> {target_cls}")

        if target_cls == 'NORMAL':
            normal_files.extend(images)
        else:
            pneumonia_files.extend(images)
    else:
        # Try fixing "Lung_Opacity" vs "Lung Opacity" naming issue
        if "Lung_Opacity" in path:
            alt_path = path.replace("Lung_Opacity", "Lung Opacity")
            if os.path.exists(alt_path):
                images = [os.path.join(alt_path, f) for f in os.listdir(alt_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                print(f"   Found {len(images)} images in: {alt_path} -> {target_cls}")
                pneumonia_files.extend(images)
            else:
                print(f"⚠️ WARNING: Folder not found: {path}")
        else:
            print(f"⚠️ WARNING: Folder not found: {path}")

print("-" * 40)
print(f"📊 TOTAL NORMAL FOUND: {len(normal_files)}")
print(f"📊 TOTAL PNEUMONIA FOUND: {len(pneumonia_files)}")
print(f"🔥 GRAND TOTAL: {len(normal_files) + len(pneumonia_files)}")
print("-" * 40)

# --- 3. DISTRIBUTE WITHOUT TRIMMING ---
def distribute(file_list, category):
    random.shuffle(file_list)
    total = len(file_list)
    train_end = int(0.70 * total)
    test_end = int(0.90 * total)

    print(f"🚀 Moving {total} {category} images...")

    for i, f in enumerate(tqdm(file_list)):
        if i < train_end: split = 'train'
        elif i < test_end: split = 'test'
        else: split = 'val'

        shutil.copy(f, os.path.join(BASE_DIR, split, category, os.path.basename(f)))

distribute(normal_files, 'NORMAL')
distribute(pneumonia_files, 'PNEUMONIA')

print("\n✅ SUCCESS! Full dataset prepared in 'Final_Dataset_Full'")

🕵️ SEARCHING FOR IMAGES...
   Found 0 images in: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal -> NORMAL
   Found 0 images in: covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity -> PNEUMONIA
   Found 0 images in: covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia -> PNEUMONIA
   Found 0 images in: covid19-radiography-database/COVID-19_Radiography_Dataset/COVID -> PNEUMONIA
   Found 1341 images in: chest-xray-pneumonia/chest_xray/train/NORMAL -> NORMAL
   Found 3875 images in: chest-xray-pneumonia/chest_xray/train/PNEUMONIA -> PNEUMONIA
   Found 234 images in: chest-xray-pneumonia/chest_xray/test/NORMAL -> NORMAL
   Found 390 images in: chest-xray-pneumonia/chest_xray/test/PNEUMONIA -> PNEUMONIA
   Found 8 images in: chest-xray-pneumonia/chest_xray/val/NORMAL -> NORMAL
   Found 8 images in: chest-xray-pneumonia/chest_xray/val/PNEUMONIA -> PNEUMONIA
----------------------------------------
📊 TOTAL NORMAL FOUND: 1583
📊 TOTAL PN

100%|██████████| 1583/1583 [00:12<00:00, 130.19it/s]


🚀 Moving 4273 PNEUMONIA images...


100%|██████████| 4273/4273 [00:01<00:00, 2751.09it/s]


✅ SUCCESS! Full dataset prepared in 'Final_Dataset_Full'


In [7]:
# ==========================================
# FINAL FIX: AUTO-DISCOVERY MERGE (SMART SCAN)
# ==========================================
import os
import shutil
import random
from tqdm import tqdm

BASE_DIR = 'Final_Dataset_Full'
if os.path.exists(BASE_DIR): shutil.rmtree(BASE_DIR)

# Create destination folders
for split in ['train', 'test', 'val']:
    for cls in ['NORMAL', 'PNEUMONIA']:
        os.makedirs(os.path.join(BASE_DIR, split, cls), exist_ok=True)

print("🕵️ STARTING DEEP SCAN FOR ALL IMAGES...")

# Lists to hold file paths
normal_files = []
pneumonia_files = []

# We walk through every single folder in your Colab workspace
for root, dirs, files in os.walk('.'):
    # Skip the destination folder itself to avoid loops
    if BASE_DIR in root or '.config' in root:
        continue

    folder_name = os.path.basename(root).lower()

    # 1. IDENTIFY PNEUMONIA FOLDERS
    # We look for keywords: 'covid', 'lung_opacity', 'viral pneumonia', or just 'pneumonia'
    if folder_name in ['covid', 'lung_opacity', 'lung opacity', 'viral pneumonia', 'pneumonia']:
        # Found a sickness folder! Add images to Pneumonia list
        images = [os.path.join(root, f) for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if len(images) > 0:
            print(f"   ✅ Found PNEUMONIA data: {len(images)} images in '{root}'")
            pneumonia_files.extend(images)

    # 2. IDENTIFY NORMAL FOLDERS
    elif folder_name == 'normal':
        # Found a healthy folder! Add images to Normal list
        images = [os.path.join(root, f) for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if len(images) > 0:
            print(f"   ✅ Found NORMAL data:    {len(images)} images in '{root}'")
            normal_files.extend(images)

print("-" * 40)
print(f"📊 TOTAL NORMAL FOUND:    {len(normal_files)}")
print(f"📊 TOTAL PNEUMONIA FOUND: {len(pneumonia_files)}")
print(f"🔥 GRAND TOTAL:           {len(normal_files) + len(pneumonia_files)}")
print("-" * 40)

if len(normal_files) < 5000:
    print("⚠️ WARNING: Still low on images. Did you run the 'Download' step?")
else:
    print("🚀 SUCCESS! Starting merge...")

# --- DISTRIBUTE FILES ---
def distribute(file_list, category):
    random.shuffle(file_list)
    total = len(file_list)
    train_end = int(0.70 * total)
    test_end = int(0.90 * total)

    for i, f in enumerate(tqdm(file_list)):
        if i < train_end: split = 'train'
        elif i < test_end: split = 'test'
        else: split = 'val'

        # Handle duplicate filenames by renaming if necessary
        dest_path = os.path.join(BASE_DIR, split, category, os.path.basename(f))
        if os.path.exists(dest_path):
            filename, ext = os.path.splitext(os.path.basename(f))
            dest_path = os.path.join(BASE_DIR, split, category, f"{filename}_dup{i}{ext}")

        shutil.copy(f, dest_path)

distribute(normal_files, 'NORMAL')
distribute(pneumonia_files, 'PNEUMONIA')

print("\n✅ FULL DATASET READY IN 'Final_Dataset_Full'")

🕵️ STARTING DEEP SCAN FOR ALL IMAGES...
   ✅ Found PNEUMONIA data: 1102 images in './Final_Dataset/train/PNEUMONIA'
   ✅ Found NORMAL data:    1102 images in './Final_Dataset/train/NORMAL'
   ✅ Found PNEUMONIA data: 158 images in './Final_Dataset/val/PNEUMONIA'
   ✅ Found NORMAL data:    158 images in './Final_Dataset/val/NORMAL'
   ✅ Found PNEUMONIA data: 315 images in './Final_Dataset/test/PNEUMONIA'
   ✅ Found NORMAL data:    315 images in './Final_Dataset/test/NORMAL'
   ✅ Found PNEUMONIA data: 3875 images in './chest-xray-pneumonia/chest_xray/train/PNEUMONIA'
   ✅ Found NORMAL data:    1341 images in './chest-xray-pneumonia/chest_xray/train/NORMAL'
   ✅ Found PNEUMONIA data: 3875 images in './chest-xray-pneumonia/chest_xray/__MACOSX/chest_xray/train/PNEUMONIA'
   ✅ Found NORMAL data:    1341 images in './chest-xray-pneumonia/chest_xray/__MACOSX/chest_xray/train/NORMAL'
   ✅ Found PNEUMONIA data: 8 images in './chest-xray-pneumonia/chest_xray/__MACOSX/chest_xray/val/PNEUMONIA'
   ✅

100%|██████████| 14394/14394 [00:12<00:00, 1175.44it/s]


✅ FULL DATASET READY IN 'Final_Dataset_Full'


In [11]:
# ==========================================
# STEP 3: TRAIN ON FULL CLEAN DATASET (15k Images)
# ==========================================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = 'Final_Dataset_Full' # Pointing to the clean, merged dataset

print(f"🚀 Preparing to train on cleaned dataset: {DATA_DIR}")

# 1. DATA GENERATORS
# Augmentation helps the model generalize better on this large dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    f'{DATA_DIR}/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    f'{DATA_DIR}/val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# 2. BUILD ROBUST CNN MODEL
model = Sequential([
    # Block 1
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Block 2
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Block 3
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Block 4 (Deep Features)
    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Classification Head
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5), # Prevents overfitting
    Dense(1, activation='sigmoid') # Binary Output (0=Normal, 1=Pneumonia)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. START TRAINING
# Since we have ~15,000 images, 5 Epochs is plenty to get high accuracy without waiting hours.
print("🔥 Starting Training... (This will take approx 20-30 mins)")

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

# 4. SAVE MODEL
model.save('pneumonia_mega_model.h5')
print("\n✅ SUCCESS! Model saved as 'pneumonia_mega_model.h5'")

🚀 Preparing to train on cleaned dataset: Final_Dataset_Full
Found 10373 images belonging to 2 classes.
Found 1482 images belonging to 2 classes.
🔥 Starting Training... (This will take approx 20-30 mins)
Epoch 1/5
325/325 ━━━━━━━━━━━━━━━━━━━━ 239s 711ms/step - accuracy: 0.8284 - loss: 1.6279 - val_accuracy: 0.6788 - val_loss: 18.7159
Epoch 2/5
325/325 ━━━━━━━━━━━━━━━━━━━━ 217s 668ms/step - accuracy: 0.9079 - loss: 0.2412 - val_accuracy: 0.6862 - val_loss: 3.6997
Epoch 3/5
325/325 ━━━━━━━━━━━━━━━━━━━━ 216s 662ms/step - accuracy: 0.9124 - loss: 0.2273 - val_accuracy: 0.9231 - val_loss: 0.2042
Epoch 4/5
325/325 ━━━━━━━━━━━━━━━━━━━━ 214s 660ms/step - accuracy: 0.9316 - loss: 0.2070 - val_accuracy: 0.6788 - val_loss: 13.5913
Epoch 5/5
325/325 ━━━━━━━━━━━━━━━━━━━━ 214s 659ms/step - accuracy: 0.9282 - loss: 0.2007 - val_accuracy: 0.8900 - val_loss: 0.3023



✅ SUCCESS! Model saved as 'pneumonia_mega_model.h5'


In [13]:
import tensorflow
import numpy
import PIL # This is Pillow
import opendatasets
import tqdm
import matplotlib

print("Copy these lines below:")
print("-" * 30)
print(f"tensorflow=={tensorflow.__version__}")
print(f"numpy=={numpy.__version__}")
print(f"Pillow=={PIL.__version__}")
print(f"opendatasets=={opendatasets.__version__}")
print(f"tqdm=={tqdm.__version__}")
print(f"matplotlib=={matplotlib.__version__}")
print("flask") # Flask is standard, version doesn't strictly matter
print("-" * 30)

Copy these lines below:
------------------------------
tensorflow==2.19.0
numpy==2.0.2
Pillow==11.3.0
opendatasets==0.1.22
tqdm==4.67.1
matplotlib==3.10.0
flask
------------------------------
